# qemu



## omg this is complicated but still better than virtualbox




### mac

https://github.com/kholia/OSX-KVM




#### mac config


In [ ]:
qemu-system-x86_64 -vga std -nographic -vnc :1 -enable-kvm -m 3072 -cpu Penryn,kvm=on,vendor=GenuineIntel,+invtsc,vmware-cpuid-freq=on,$MY_OPTIONS\
          -machine pc-q35-2.11 \
          -smp 4,cores=2 \
          -usb -device usb-kbd -device usb-tablet \
          -device isa-applesmc,osk="ourhardworkbythesewordsguardedpleasedontsteal(c)AppleComputerInc" \
          -drive if=pflash,format=raw,readonly,file=OVMF_CODE.fd \
          -drive if=pflash,format=raw,file=OVMF_VARS-1024x768.fd \
          -smbios type=2 \
          -device ich9-intel-hda -device hda-duplex \
          -device ide-drive,bus=ide.2,drive=Clover \
          -drive id=Clover,if=none,snapshot=on,format=qcow2,file=./'Mojave/Clover.qcow2' \
          -device ide-drive,bus=ide.1,drive=MacHDD \
          -drive id=MacHDD,if=none,file=./mac_hdd.img,format=qcow2 \
          -device ide-drive,bus=ide.0,drive=MacDVD \
          -drive id=MacDVD,if=none,snapshot=on,media=cdrom,file=./'Mojave.iso' \
          -netdev tap,id=net0,ifname=tap0,script=no,downscript=no -device vmxnet3,netdev=net0,id=net0,mac=52:54:00:AB:F8:B7
          
          

### ubuntu



#### working ubuntu config

https://www.dzombak.com/files/qemu-bridging-mavericks.pdf

```
    -net nic,model=virtio,macaddr=54:54:00:55:55:55 \
    -netdev tap,ifname=tap0,script=./tap-up.sh,downscript=./tap-down.sh,id=n1 \
    -device e1000-82545em,netdev=n1
```


In [ ]:
qemu-system-x86_64 -monitor stdio \
    -device ati-vga \
    -enable-kvm -accel hvf \
    -smp 2 \
    -m 3G \
    -netdev tap,ifname=tap0,br=bridge1,script=no,downscript=no,id=n1 \
    -device rtl8139,netdev=n1 \
    -drive if=ide,index=1,media=cdrom,file="" \
    -drive file="./VirtualBox VMs/Ubuntu/Ubuntu.vmdk",if=none,id=ubuntuhdd \
    -device virtio-blk-pci,drive=ubuntuhdd,bus=pci.0,bootindex=2 \
    -device piix3-usb-uhci \
    -drive id=freeagent,file=/dev/disk2,if=none,format=raw \
    -device usb-storage,drive=freeagent

#### more configs



In [ ]:
qemu-system-x86_64 -smp 3 -cpu host -m 3G -vga qxl \
    -spice port=5924,disable-ticketing \
    -drive file=/Users/briancullinan/VirtualBox\ VMs/Ubuntu/Ubuntu.vmdk,if=virtio \
    -net nic,model=virtio -net user,hostname=ubuntuvm \
    -monitor stdio -name "Ubuntu" \
&& spicy --title Ubuntu 127.0.0.1 -p 5924

### windows


#### more info





Guest
During installation at the partition step Windows doesn't detect the VirtIO hard drive. You have to tell Windows to use the viostor driver from the driver image.
After installation Windows doesn't for the VirtIO ethernet adapter. You have to tell Windows to use the netkvm driver from the driver image.
For 32 bit Windows 7 a sound driver for the Intel 82801AA AC97 exists.
For 64 bit Windows 7 Intel HDA is available as an option (QEMU option: -soundhw hda)
USB 2.0 pass through can be configured from host to guest with variations of: -usb -device usb-ehci,id=ehci -device usb-host,bus=ehci.0,vendorid=1452
For Windows 8.1 USB tablet is available only with USB 2.0 pass through (QEMU option: -device usb-ehci,id=ehci -device usb-tablet,bus=ehci.0
SPICE
Running SPICE
SPICE (KVM)
QEMU with SPICE support enable (among other things) the more powerful QXL display device and makes clipboard sharing possible (copy/paste between clients and the virtual machine).

To use SPICE with QEMU enable the following USE flag in package.use:

FILE /etc/portage/package.useSetting USE variable
app-emulation/qemu   spice


#### working windows config



In [ ]:
qemu-system-x86_64 -monitor stdio \
    -device ati-vga \
    -enable-kvm -accel hvf \
    -smp 2 \
    -m 3G \
    -net nic,model=virtio -net user,hostname=windowsvm \
    -drive if=ide,index=1,media=cdrom,file="" \
    -drive file="./Parallels/Windows 10.pvm/harddisk.hdd/harddisk.hdd.0.{5fbaabe3-6958-40ff-92a7-860e329aab41}.hds",if=none,id=windowshdd \
    -device virtio-blk-pci,drive=windowshdd,bus=pci.0,bootindex=2


#### more configs



In [ ]:
qemu-system-x86_64 --enable-kvm \ 
-drive file=./win10.img,if=virtio \
-smp 4 -m 4096 -vga qxl \
-net nic,model=virtio -net user \
-soundhw hda

qemu-system-x86_64 --enable-kvm \
-drive file=./win10.img,if=virtio \
-smp 4 -cpu host -m 4096 -vga qxl \
-net nic,model=virtio -net user \
-soundhw hda


In [ ]:
#!/bin/sh
SPICE_PORT=5924
qemu-system-x86_64 -enable-kvm -daemonize \
    -cpu host \
    -drive file=WindowsVM.img,if=virtio \
    -net nic -net user,hostname=windowsvm \
    -m 1G \
    -vga qxl \
    -spice port=${SPICE_PORT},disable-ticketing \
    -usbdevice tablet \
    -device virtio-serial \
    -chardev spicevmc,id=vdagent,name=vdagent \
    -device virtserialport,chardev=vdagent,name=com.redhat.spice.0 \
    "$@"
exec spicy --title Windows 127.0.0.1 -p 5924



In [ ]:
#!/bin/sh
SPICE_PORT=5924
qemu-system-x86_64 -enable-kvm -daemonize \
    -cpu host \
    -drive file=WindowsVM.img,if=virtio \
    -net nic -net user,hostname=windowsvm \
    -m 1G \
    -vga qxl \
    -spice port=${SPICE_PORT},disable-ticketing \
    -usbdevice tablet \
    -device virtio-serial \
    -chardev spicevmc,id=vdagent,name=vdagent \
    -device virtserialport,chardev=vdagent,name=com.redhat.spice.0 \
    "$@"
exec remote-viewer --title Windows spice://127.0.0.1:${SPICE_PORT}



In [ ]:
#!/bin/sh
WINIMG=~/vm/Win10_1803_English_x64.iso
VIRTIMG=~/vm/virtio-win-0.1.149.iso
qemu-system-x86_64 --enable-kvm -drive driver=raw,file=~/vm/win10.img,if=virtio -m 6144 \
-net nic,model=virtio -net user -cdrom ${WINIMG} \
-drive file=${VIRTIMG},index=3,media=cdrom \
-rtc base=localtime,clock=host -smp cores=4,threads=8 \
-usb -device usb-tablet \
-net user,smb=$HOME

